## Lab #1 - Sandbox notebook for the use case: Courier default

Let's assume we are all data-scientists, and we are currently working on the exploration and development of our models for solving the courier default use case.

-------

**Courier default description:**

*"We define courier default when a courier collects a high amount of cash and then leaves the company without returning the money.

- By leaving the company we define as not having any interaction with us during 28 consecutive days since the last delivered order.
- Each region/city/country has a threshold to designate a high amount of cash defined by the business.

The business request made to the Data Science team is to design a machine learning system to detect when a courier is likely to incur in default before they actually default or before the amount of cash balance is too high.

The proposed definition is to deliver a default score for each courier every week. For that we created a label for each courier on a window time, so we can identify some defaulters and then fit a xgboost classifier to predict on current couriers."*

-------

In this notebook we will explore different methods for working with this binary classification use case based on XGBoost, in order to predict the default or not default condition for the couriers.

Let's try with these approaches:

1. We will load the existing XGBoost model file (trained outside of AWS), and analyse it with open-source XGBoost.
2. We will repeat the analysis levaraging on Amazon SageMaker Autopilot for using AutoML.
3. Finally, we will repeat the step 1 but leveraging on Amazon SageMaker built-in XGBoost algorithm, and we will use the hyperparameters from step 2.

#### Importing relevant packages...

Let's start by loading some relevant libraries.

In [ ]:
## Importing relevant packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import os
import datetime as dt
import pickle as pkl
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

import sagemaker
import boto3
from sagemaker.predictor import csv_serializer
from sagemaker import get_execution_role

In [ ]:
!conda install -y -c conda-forge xgboost==0.90

### 1. Working with the existing model

Let's work with the dataset provided, and perform the same feature selection done as per the use case description.

**IMPORTANT:**
Make sure you get the dataset and the model files from the workshop organizers (joint_dataframe.csv & model-sav), and upload it to your Jupyter environment by choosing "Upload".


In [ ]:
## Read data

import xgboost as xgb
import json
from sklearn.model_selection import train_test_split

data = pd.read_csv('joint_dataframe.csv')

data.head(5)

In [ ]:
### Store the target feature in a separate dataframe

y_data = data['is_defaulter']
print(y_data.shape)

In [ ]:
### Drop some features as per the use case description

cols = [0, 1, 2, 3, 4, 5, 6, 8, 9, 56, 191, 201, 203, 206, 209, 218, 220]
data.drop(data.columns[cols],axis=1,inplace=True)
data.head(5)

In [ ]:
### Split the dataset in train/test, and convert the dataframes to XGBoost matrixes

X = data.values
Y = y_data.values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25)

D_train = xgb.DMatrix(X_train, label=Y_train)
D_test = xgb.DMatrix(X_test, label=Y_test)

print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

Now, let's load the existing model trained outside AWS.

In [ ]:
## Load existing model

model = pkl.load(open("model-sav", "rb"))
type(model)


Let's run predictions over the testing dataset, and evaluate some model metrics.

In [ ]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, accuracy_score

preds = model.predict(X_test)
best_preds = np.asarray([np.argmax(line) for line in preds])

print("Precision = {}".format(precision_score(Y_test, best_preds, average='macro')))
print("Recall = {}".format(recall_score(Y_test, best_preds, average='macro')))
print("Accuracy = {}".format(accuracy_score(Y_test, best_preds)))


The model seems to have decent accuracy but the precision and recall seems to be low.

Let's confirm the confusion matrix to understand how much false negatives and false positives we are having.

In [ ]:
cm = confusion_matrix(Y_test, (preds>0.5))
print('The Confusion Matrix is: \n', cm)
print('TrueNegative   FalsePositive\nFalseNegative   TruePositive\n')

# Calculate the accuracy on test set
predict_accuracy_on_test_set = (cm[0,0] + cm[1,1])/(cm[0,0] + cm[1,1]+cm[1,0] + cm[0,1])
print('The Accuracy on Test Set is: ', predict_accuracy_on_test_set)

In [ ]:
print("Indices where is_defaulter should be one: ", np.nonzero(Y_test > 0))
print("\nIndices where is_defaulter is one: ", np.nonzero(preds > 0))

print("\n------- False negative case: -------")
print("Defaulter data: \n", list(X_test[3][:]))
print("The true is_defaulter value: ", Y_test[3], " the predicted is_defaulter value: ", preds[3])
sample = [60.11, 110.0, 0.99, 0.09, 5.0, 0.77, 0.44, 0.5, 0.36, 2.29, 0.01, 48.0, 0.02, 1.0, 110.0, 0.64, 1.01, 0.71, 0.00495662949194547, 4.0, 0.02, 0.98, 807.0, 1.0, 4.0, 111.0, 1.33, 0.44, 0.0, 5.0, 11.0, 0.5128205128205128, 0.0, 39.0, 2.0, 0.18181818181818185, 0.0924170616113744, 0.0196078431372549, 0.07471264367816091, 39.0, 1.0, 0.0, 0.0, 0.8478260869565217, 0.96, 0.38, 31.43, 0.05, 62.46, 3.21252782376189, 60.11, 2.0, 0.7, 37.0, 1.297491039426523, 47.0, 213.0, 38.0, 753.0, 1.027027027027027, 181.0, 0.7926315789473685, 0.5046480743691899, 1.0734463276836157, 0.5498007968127491, 147.0, 0.668086239020495, 414.0, 0.694555112881806, 1.2312925170068028, 523.0, 0.8497652582159625, 10.02, 0.46, 10.43, 1172.27, 189.4, 0.0, 2.3, 0.0, 2196.57, 1.0, 0.0, 0.04, 211355.43, 84.48, 1.91, 0.0, 59.3, 0.0, 1.03, 0.7, 0.01, 0.02, 298.17, 0.689604842322592, 0.0, 227.86, 0.0, 52.07, 1.94, 0.02, 0.9, 395913.79, 0.31, 0.0, 1.28, 12.71, 83.74, 765.42, 0.1, 579.95, 0.01, 1.0, 8604.43, 130.06, 0.54, 70.1718644837339, 0.0, 0.0, 0.0, 0.56, 24.0, 9.0, 0.66, 0.0, 0.0, 19.0, 127.230466482265, 85.90011414995679, 55.0, 0.16, 1.07, 0.0, 71.6590049396513, 0.26, 0.0, 42.08, 0.0, 0.0, 37.45, 0.0, 1.08, 0.0, 10.0, 37.96, 90.0, 0.0, 19.0, 0.0, 55.0, 0.5, 28.0, 0.37, 0.0, 0.31, 40.0, 90.0, 0.16, 40.45, 38.0, 27.0, 173.0, 0.0, 178.0, 0.41, 0.24, 0.0, 0.49, 0.0, 0.0, 0.51, 46.57, 28.0, 0.32, 0.0, 0.0, 90.8904313776849, 0.52, 0.93, 1.0, 7.0, 0.0, 36.9236331328595, 0.0, 825.0, 0.0, 1.0, 1.0, 0.0, 1.53846153846154, 26.23, 0.0, 2.4935483870967703, 0.0, 6.125, 27.0, 753.0, 7.0, 83.0, 0.0, 1.5609320122090802, 21.6833, 310.0, 0.46524609707560494, 143.92, 13.0, 0.0, 0.91, 0.24, 21.0, 380.0, 1.65, 1.77, 80.0, 5.0, 41.0, 0.45, 0.45, 36.0, 0.89, 57.4, 0.41, 0.57, 0.1192, 0.94113029827315, 232.33333333333331, 23.98, 0.0784, 10.5, 11.08, 0.8849926199200001, 0.78, 28.5, 6.97, 27.37, 27.2, 0.0491, 0.39, 0.1769, 2.87, 0.022370486656200003, 0.4, 0.73, 0.57, 24.33333333333333, 28.0, 0.15, 0.71, 12.0, 18.0, 12.0, 2.33, 9.0, 0.13, 0.74, 100000.0, 0.0, 1.77, 0.13, 0.22, 0.0, 0.3193277310924369, 950.0, 0.0, 0.2689075630252101, 0.0, 0.9, 0.22, 10.0, 0.56, 0.6956, 10000.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.5384, 689.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.54, 0.0, 10000.0, 0.0, 4.0, 1.54, 1.0, 0.8571, 0.0, 1.0, 0.0, 1.4, 1.54, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.2, 0.0, 0.0, 1.0, 360.0, 24.0, 0.8343309106455751, 288.0, 48.0, 0.8379373848987108]
pred = model.predict(sample)
print(pred)

print("\n------- False positive case: -------")
print("Defaulter data: \n", list(X_test[1886][:]))
print("The true is_defaulter value: ", Y_test[1886], " the predicted is_defaulter value: ", preds[1886])
sample = [184.0, 2900.0, 0.98, 0.39, 5.0, 0.0, 0.51, 0.5, 4.21, 1.78, 0.02, 23.0, 0.12, 71.0, 40.85, 0.65, 1.02, 0.0, 0.00825206301575393, 4.0, 0.04, 0.96, 3999.0, 1.0, 33.0, 2971.0, 0.0, 0.58, 7.142857142857142, 1.0, 3.0, 6.7857142857142865, 5.0, 180.0, 19.0, 6.333333333333332, 0.03443657929978955, 0.04231625835189309, 0.030349013657056136, 127.0, 0.7055555555555556, 5.0, 0.04273504273504273, 6.143344709897611, 0.07, 1.89, 145.4, 0.77, 2599.0, 1038.0, 184.0, 12.0, 0.49, 20.0, 4.235171696149844, 110.0, 443.0, 99.0, 5422.0, 4.95, 407.0, 0.9366039039557782, 0.2816303946883069, 3.928571428571429, 0.3865732202139432, 132.0, 7.147376746638543, 2096.0, 0.6099225378089266, 3.083333333333333, 3307.0, 0.9187358916478556, 7.12, 4.54, 17.06, 4781.02, 438.94, 0.0, 1.45, 0.0, 22482.46, 1.0, 0.89, 0.0, 263691.34, 277.56, 0.92, 0.0, 339.35, 3.9586415365578294e-05, 3.4, 1.22, 0.0, 0.0, 1015.04, 0.0, 0.0, 379.52, 0.0, 0.0, 0.32, 0.0, 7.12, 1077023.92, 0.77, 0.0, 3.31, 0.0, 1225.2, 0.0, 0.02, 436.29, 0.0, 0.0, 95907.23, 74.76, 0.59, 41.322943430657, np.nan, 0.0, np.nan, 0.72, np.nan, np.nan, 0.82, 0.0, 0.0, np.nan, 60.1107199324761, 50.785041042695, np.nan, 0.0, 1.42, np.nan, 48.1914191364083, 0.0, 0.0, 32.95, 0.0, 0.0, 25.9, 0.0, 0.89, 0.0, np.nan, 24.37, np.nan, 0.0, np.nan, np.nan, np.nan, 0.0, np.nan, 0.57, np.nan, 0.0, np.nan, np.nan, 0.0, 34.5, np.nan, np.nan, np.nan, np.nan, np.nan, 0.52, 0.0, np.nan, 0.65, 0.0, np.nan, 0.0, 34.03, np.nan, 0.0, 0.0, 0.0, 49.9810437910087, 0.0, 1.05, 0.0, 1.0, 0.0, 100.0, 0.0, 5622.0, 0.0, 1.0, 1.0, 3.0, 1.35, 18.46, 0.0, 10.0, 0.0, 35.0, 50.0, 5422.0, 1.0, 10.0, 0.0, 1.2351362605839098, 35.3166, 9.0, 1.0377370041887801, 4.94, 83.0, 20.0, 4.65, 0.0, 0.0, 1527.0, 0.0, 0.0, 0.0, 37.0, 12.0, 4.44, 0.0, 136.0, 6.45, 18.15, 0.52, 0.35, 1.005, 0.86319050077439, 234.0, 16.72, 0.5118, 12.99, 12.68, 1.1144399999999999, 0.18, 16.6, 7.02, 21.89, 17.81, 0.6659999999999999, 0.73, 0.602, 3.63, 0.018069179143000002, 0.58, 4.6, 0.83, 153.33333333333334, 13.0, 0.02, 6.76, 0.0, 3.0, 196.0, 130000.0, 127.0, 0.03, 0.52, 30000.0, 0.0, 0.7, 0.03, 0.03, 0.0, 0.14545454545454545, 5789.0, 0.0, 0.2545454545454545, 0.0, 0.0, 0.1, 3.0, 0.6521, 0.8571, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 10000.0, 1.0, 1.0, 10000.0, 0.84, 0.0, 1.0, 0.0, 1.0, 1.5384, 3909.0, 33.33, 1.0, 0.0, 8.33, 0.0, 0.0, 5.56, 0.0, 0.0, 7.0, 0.0, 0.0, 1.1538, 0.0, 2.0, 10000.0, 1.5, 4.88, 0.0, 1.0, 0.14, 0.0, 0.0, 0.0, 1.2857, 0.0, 10000.0, 0.0, 2092.0, 141.0, 0.9225806451612903, 4995.0, 126.0, 0.41353248155479577]
pred = model.predict(sample)
print(pred)

## 2. Now, let's try with Amazon SageMaker built-in XGBoost, but with the same feature selection as per Glovo

We have seen a method for working with the example use case provided. Now, let's explore how could we replicate the case using the Amazon SageMaker built-in XGBoost algorithm.

In this case we will just perform the same feature selection as per the previous case, but we will consider some hyperparameters adjustments. It should result on just a slightly better model.

In [ ]:
### Set some parameters for starting

print('Amazon SageMaker version:', sagemaker.__version__)

region = boto3.Session().region_name
session = sagemaker.Session()
role = get_execution_role()
sm = boto3.Session().client(service_name='sagemaker', region_name=region)

print('Using region:', region,'\nUsing role:',role)

###########################################################
##### REPLACE with the bucket name for your S3 bucket #####
###########################################################
bucket = 'YOUR BUCKET NAME GOES HERE'
prefix = 'glovo/sandbox/builtin'

print('Using bucket and prefix: {}/{}'.format(bucket, prefix))

data = pd.read_csv('joint_dataframe.csv')
data.head(5)

In [ ]:
cols = [0, 1, 2, 3, 4, 5, 6, 8, 56, 191, 201, 203, 206, 209, 218, 220]
data.drop(data.columns[cols],axis=1,inplace=True)
#data.head(5)

In [ ]:
import os

data = pd.concat([data['is_defaulter'], data.drop(['is_defaulter'], axis=1)], axis=1) 

train_data, validation_data, test_data = np.split(data.sample(frac=1, random_state=1729), [int(0.7 * len(data)), int(0.95 * len(data))])
train_data.to_csv('train-built.csv', header=False, index=False)
validation_data.to_csv('validation-built.csv', header=False, index=False)
test_data.to_csv('test_real-built.csv', header=False, index=False)
test_data.drop(['is_defaulter'], axis=1).to_csv('test-built.csv', header=False, index=False)

boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/train-built.csv')).upload_file('train-built.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation/validation-built.csv')).upload_file('validation-built.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'test/test-built.csv')).upload_file('test-built.csv')

In [ ]:
s3_input_train = sagemaker.s3_input(s3_data='s3://{}/{}/train'.format(bucket, prefix), content_type='csv')
s3_input_validation = sagemaker.s3_input(s3_data='s3://{}/{}/validation'.format(bucket, prefix), content_type='csv')
s3_input_test = sagemaker.s3_input(s3_data='s3://{}/{}/test'.format(bucket, prefix), content_type='csv')

Now, with the split data in S3 and ready for training, we will setup an Amazon SageMaker XGBoost training job.

In [ ]:
containers = {'eu-west-1': '685385470294.dkr.ecr.eu-west-1.amazonaws.com/xgboost:latest'}

xgbb = sagemaker.estimator.Estimator(containers[region],
                                    role,
                                    train_instance_count=1, 
                                    train_instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=session)
xgbb.set_hyperparameters(objective='binary:logistic',
                         alpha = 0.0017958870152480393,
                         colsample_bytree = 0.8974444697232986,
                         eta = 0.378416419404957,
                         gamma = 0.0038479366336815115,
                         max_depth = 22,
                         min_child_weight = 3.4445863514152535,
                         num_round = 139,
                         subsample = 0.7432022124726009
                        )

Let's train this model.

In [ ]:
xgbb.fit({'train': s3_input_train, 'validation': s3_input_validation})

After around 5 minutes we are ready to deploy this model on an endpoint for inference.

In [ ]:
xgbb_predictor = xgbb.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

After around 6 minutes we have an Endpoint ready to serve inferences.
Let's run the test data inferences on the endpoint, and evaluate the model.

In [ ]:
xgbb_predictor.content_type = 'text/csv'
xgbb_predictor.serializer = csv_serializer
xgbb_predictor.deserializer = None

def predict(data, rows=500):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, xgbb_predictor.predict(array).decode('utf-8')])

    return np.fromstring(predictions[1:], sep=',')

In [ ]:
test_data_no_target = test_data.drop(test_data.columns[0],axis=1)
#test_data_no_target.head(5)

In [ ]:
#test_data.head(5)

In [ ]:
predsb = predict(test_data_no_target.values)
predsb

In [ ]:
Y_testb = test_data.values[:,0]
print(Y_testb)

In [ ]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, accuracy_score

best_predsb = np.asarray([np.argmax(line) for line in predsb])

print("Precision = {}".format(precision_score(Y_testb, best_predsb, average='macro')))
print("Recall = {}".format(recall_score(Y_testb, best_predsb, average='macro')))
print("Accuracy = {}".format(accuracy_score(Y_testb, best_predsb)))

In [ ]:
cmb = confusion_matrix(Y_testb, (predsb>0.5))
print('The Confusion Matrix is: \n', cmb)
print('TrueNegative   FalsePositive\nFalseNegative   TruePositive\n')

# Calculate the accuracy on test set
predict_accuracy_on_test_setb = (cmb[0,0] + cmb[1,1])/(cmb[0,0] + cmb[1,1]+cmb[1,0] + cmb[0,1])
print('The Accuracy on Test Set is: ', predict_accuracy_on_test_setb)

As expected, this model is a bit better than our first case, but still has a lot of room for improvement.

### 3. Finally, let's try with Amazon SageMaker Autopilot

Now, we will repeat the work but this time leveraging on AutoML. Obviously we expect better results as the previous examples were just doing a feature selection, while the AutoML should perform a full feature engineering and hyper-parameters optimization.

*For more info on Amazon SageMaker Autopilot for AutoML you can check the documentation [here](https://docs.aws.amazon.com/sagemaker/latest/dg/autopilot-automate-model-development.html), and use [this example notebook](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/autopilot/autopilot_customer_churn_high_level_with_evaluation.ipynb) as a reference.*


In [ ]:
### Set some parameters for starting

print('Amazon SageMaker version:', sagemaker.__version__)

region = boto3.Session().region_name
session = sagemaker.Session()
role = get_execution_role()
sm = boto3.Session().client(service_name='sagemaker', region_name=region)

print('Using region:', region,'\nUsing role:',role)

prefix = 'glovo/sandbox'

print('Using bucket and prefix: {}/{}'.format(bucket, prefix))

df = pd.read_csv('joint_dataframe.csv')

In [ ]:
## Prepare original data for AutoML
## This time we won't perform any feature selection as this will be done by SageMaker Autopilot

train_data = df.sample(frac=0.8,random_state=200)

test_data = df.drop(train_data.index)

test_data_no_target = test_data.drop(columns=['is_defaulter'])

train_file = 'train_data.csv'
train_data.to_csv(train_file, index=False, header=True)

test_file = 'test_data.csv'
test_data_no_target.to_csv(test_file, index=False, header=False)

In [ ]:
## Setup an Amazon SageMaker Autopilot job

### Adjust the max_candidates parameter below, this will affect the time it takes to complete the AutoML job

from sagemaker import AutoML
from time import gmtime, strftime, sleep

timestamp_suffix = strftime('%d-%H-%M-%S', gmtime())
base_job_name = 'glovo-automl-sdk-' + timestamp_suffix

target_attribute_name = 'is_defaulter'
target_attribute_values = np.unique(train_data[target_attribute_name])
target_attribute_true_value = target_attribute_values[1] # 'True.'

account_id = boto3.client('sts').get_caller_identity().get('Account')

automl = AutoML(role=role,
                target_attribute_name=target_attribute_name,
                base_job_name=base_job_name,
                sagemaker_session=session,
                max_candidates=10)

In [ ]:
### Let's train this AutoML
### Note this step should take several minutes to complete depending on the max_candidates above

automl.fit(train_file, job_name=base_job_name, wait=False, logs=False)

In [ ]:
print ('JobStatus - Secondary Status')
print('------------------------------')


describe_response = automl.describe_auto_ml_job()
print (describe_response['AutoMLJobStatus'] + " - " + describe_response['AutoMLJobSecondaryStatus'])
job_run_status = describe_response['AutoMLJobStatus']
    
while job_run_status not in ('Failed', 'Completed', 'Stopped'):
    describe_response = automl.describe_auto_ml_job()
    job_run_status = describe_response['AutoMLJobStatus']
    
    print(describe_response['AutoMLJobStatus'] + " - " + describe_response['AutoMLJobSecondaryStatus'])
    sleep(30)

In [ ]:
## Evaluate Autopilot candidates looking for the best training job

best_candidate = automl.describe_auto_ml_job()['BestCandidate']
best_candidate_name = best_candidate['CandidateName']
print(best_candidate)
print('\n')
print("CandidateName: " + best_candidate_name)
print("FinalAutoMLJobObjectiveMetricName: " + best_candidate['FinalAutoMLJobObjectiveMetric']['MetricName'])
print("FinalAutoMLJobObjectiveMetricValue: " + str(best_candidate['FinalAutoMLJobObjectiveMetric']['Value']))

In [ ]:
# Number of Autopilot candidates to evaluate and run batch transform jobs.
# Do not put a larger TOP_N_CANDIDATES than the Batch Transform limit for ml.m5.xlarge instances in your account.
TOP_N_CANDIDATES = 5

candidates = automl.list_candidates(sort_by='FinalObjectiveMetricValue',
                                    sort_order='Descending',
                                    max_results=TOP_N_CANDIDATES)

for candidate in candidates:
    print("Candidate name: ", candidate['CandidateName'])
    print("Objective metric name: ", candidate['FinalAutoMLJobObjectiveMetric']['MetricName'])
    print("Objective metric value: ", candidate['FinalAutoMLJobObjectiveMetric']['Value'])
    print('\n')

In [ ]:
#test_data.head()

In [ ]:
input_data_transform = session.upload_data(path=test_file, bucket=bucket, key_prefix=prefix)
print('Uploaded transform data to {}'.format(input_data_transform))

In [ ]:
inference_response_keys = ['predicted_label', 'probability']

In [ ]:
## Perform batch trasnformation

s3_transform_output_path = 's3://{}/{}/inference-results/'.format(bucket, prefix);

transformers = []

for candidate in candidates:
    model = automl.create_model(name=candidate['CandidateName'],
                                candidate=candidate,
                                inference_response_keys=inference_response_keys)
    
    output_path = s3_transform_output_path + candidate['CandidateName'] +'/'
    
    transformers.append(
        model.transformer(instance_count=1, 
                          instance_type='ml.m5.xlarge',
                          assemble_with='Line',
                          output_path=output_path))

print("Setting up {} Batch Transform Jobs in `transformers`".format(len(transformers)))

In [ ]:
for transformer in transformers:
    transformer.transform(data=input_data_transform, split_type='Line', content_type='text/csv', wait=False)
    print("Starting transform job {}".format(transformer._current_job_name))

In [ ]:
pending_complete = True

while pending_complete:
    pending_complete = False
    num_transform_jobs = len(transformers)
    for transformer in transformers:
        desc = sm.describe_transform_job(TransformJobName=transformer._current_job_name)
        if desc['TransformJobStatus'] not in ['Failed', 'Completed']:
            pending_complete = True
        else:
            num_transform_jobs -= 1
    print("{} out of {} transform jobs are running.".format(num_transform_jobs, len(transformers)))
    sleep(30)
    
for transformer in transformers:
    desc = sm.describe_transform_job(TransformJobName=transformer._current_job_name)
    print("Transform job '{}' finished with status {}".format(transformer._current_job_name, desc['TransformJobStatus']))

In [ ]:
## Evaluate batch inference results

import json
import io
from urllib.parse import urlparse

def get_csv_from_s3(s3uri, file_name):
    parsed_url = urlparse(s3uri)
    bucket_name = parsed_url.netloc
    prefix = parsed_url.path[1:].strip('/')
    s3 = boto3.resource('s3')
    obj = s3.Object(bucket_name, '{}/{}'.format(prefix, file_name))
    return obj.get()["Body"].read().decode('utf-8')

In [ ]:
predictions = []

for transformer in transformers:
    print(transformer.output_path)
    pred_csv = get_csv_from_s3(transformer.output_path, '{}.out'.format(test_file))
    predictions.append(pd.read_csv(io.StringIO(pred_csv), header=None))

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve, classification_report, average_precision_score
import matplotlib.pyplot as plt

labels = test_data[target_attribute_name].apply(lambda row: True if row==target_attribute_true_value else False)

# calculate auc score
for prediction, candidate in zip(predictions, candidates):
    roc_auc = roc_auc_score(labels, prediction.loc[:,1])
    ap = average_precision_score(labels, prediction.loc[:,1])
    print('%s\'s ROC AUC = %.2f, Average Precision = %.2f' % (candidate['CandidateName'], roc_auc, ap))
    print(classification_report(test_data[target_attribute_name], prediction.loc[:,0]))
    print()

In [ ]:
fpr_tpr = []
for prediction in predictions:
    fpr, tpr, _ = roc_curve(labels, prediction.loc[:,1])
    fpr_tpr.append(fpr)
    fpr_tpr.append(tpr)

plt.figure(num=None, figsize=(16, 9), dpi=160, facecolor='w', edgecolor='k')
plt.plot(*fpr_tpr)
plt.legend([candidate['CandidateName'] for candidate in candidates], loc="lower right")
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.show()

In [ ]:
precision_recall = []
for prediction in predictions:
    precision, recall, _ = precision_recall_curve(labels, prediction.loc[:,1])
    precision_recall.append(recall)
    precision_recall.append(precision)

plt.figure(num=None, figsize=(16, 9), dpi=160, facecolor='w', edgecolor='k')
plt.plot(*precision_recall)
plt.legend([candidate['CandidateName'] for candidate in candidates], loc="lower left")
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.show()

In [ ]:
target_min_precision = 0.75

best_recall = 0
best_candidate_idx = -1
best_candidate_threshold = -1
candidate_idx = 0
for prediction in predictions:
    precision, recall, thresholds = precision_recall_curve(labels, prediction.loc[:,1])
    threshold_idx = np.argmax(precision>=target_min_precision)
    if recall[threshold_idx] > best_recall:
        best_recall = recall[threshold_idx]
        best_candidate_threshold = thresholds[threshold_idx]
        best_candidate_idx = candidate_idx
    candidate_idx += 1

print("Best Candidate Name: {}".format(candidates[best_candidate_idx]['CandidateName']))
print("Best Candidate Threshold (Operation Point): {}".format(best_candidate_threshold))
print("Best Candidate Recall: {}".format(best_recall))

In [ ]:
prediction_default = predictions[best_candidate_idx].loc[:,0] == target_attribute_true_value
prediction_updated = predictions[best_candidate_idx].loc[:,1] >= best_candidate_threshold

# compare the updated predictions to Autopilot's default
from sklearn.metrics import precision_score, recall_score
print("Default Operating Point: recall={}, precision={}".format(recall_score(labels, prediction_default), precision_score(labels, prediction_default)))
print("Updated Operating Point: recall={}, precision={}".format(recall_score(labels, prediction_updated), precision_score(labels, prediction_updated)))

In [ ]:
## Deploy the selected candidate on an endpoint

inference_response_keys

In [ ]:
from sagemaker.predictor import RealTimePredictor
from sagemaker.content_types import CONTENT_TYPE_CSV

predictor = automl.deploy(initial_instance_count=1,
                          instance_type='ml.m5.xlarge',
                          candidate=candidates[best_candidate_idx],
                          inference_response_keys=inference_response_keys,
                          predictor_cls=RealTimePredictor)
predictor.content_type = CONTENT_TYPE_CSV

print("Created endpoint: {}".format(predictor.endpoint))

In [ ]:
best_candidate_threshold

In [ ]:
## Write test_data removing target 'is_defaulter'

with open('test_data_no_target.csv', 'w') as csv_file:
    test_data_no_target.to_csv(sep=',', header=False, index=False, path_or_buf=csv_file)
    

In [ ]:
from io import StringIO

prediction = predictor.predict(test_data_no_target.to_csv(sep=',', header=False, index=False)).decode('utf-8')
prediction_df = pd.read_csv(StringIO(prediction), header=None, names=inference_response_keys)
custom_predicted_labels = prediction_df.iloc[:,1].values >= best_candidate_threshold
prediction_df['custom_predicted_label'] = custom_predicted_labels
prediction_df['custom_predicted_label'] = prediction_df['custom_predicted_label'].map({False: target_attribute_values[0], True: target_attribute_values[1]})
prediction_df

In [ ]:
prediction_df_defaulters = prediction_df[prediction_df["custom_predicted_label"]==1]
prediction_df_defaulters

In [ ]:
cm = confusion_matrix(test_data['is_defaulter'], (predictions[best_candidate_idx].loc[:,0]>0.5))
print('The Confusion Matrix is: \n', cm)
print('TrueNegative   FalsePositive\nFalseNegative   TruePositive\n')

# Calculate the accuracy on test set
predict_accuracy_on_test_set = (cm[0,0] + cm[1,1])/(cm[0,0] + cm[1,1]+cm[1,0] + cm[0,1])
print('The Accuracy on Test Set is: ', predict_accuracy_on_test_set)

As you can see the AutoML provided a way better result for capturing those true positives, and the accuracy on the test set is also higher.

Note Amazon SageMaker Autopilot works with SageMaker pipelines, where it chains different container images in sequence (e.g. Pre-processing container + XGBoost classifier container) for performing the inferences directly with the original data.

In the next lab, we will learn how to industrialize/automate one of these examples following AWS MLOps best practices. We will choose the second case for simplicity and saving time, but note we could also use the AutoML artifacts from the final case if we want to.

In [ ]:
## Cleanup

# s3 = boto3.resource('s3')
# s3_bucket = s3.Bucket(bucket)

# s3_bucket.objects.filter(Prefix=prefix).delete()

# for transformer in transformers:
#     transformer.delete_model()

# predictor.delete_endpoint()
# predictor.delete_model()